In [119]:
import numpy as np

# Part 0

In [120]:

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))



In [121]:
def f(x, w, b):
    w11, w12, w21, w22, w31, w32 = w
    b1, b2, b3 = b
    
    z1 = w11*x[0] + w12*x[1] + b1
    z2 = w21*x[0] + w22*x[1] + b2
    a1 = sigmoid(z1)
    a2 = sigmoid(z2)
    z3 = w31*a1 + w32*a2 + b3
    return sigmoid(z3)



In [122]:
def gradient_f(x, w, b):
    w11, w12, w21, w22, w31, w32 = w
    b1, b2, b3 = b
    
    z1 = w11*x[0] + w12*x[1] + b1
    z2 = w21*x[0] + w22*x[1] + b2
    a1 = sigmoid(z1)
    a2 = sigmoid(z2)
    z3 = w31*a1 + w32*a2 + b3
    
    df_dz3 = sigmoid_derivative(z3)
    dz3_dw31 = a1
    dz3_dw32 = a2
    dz3_db3 = 1
    
    da1_dz1 = sigmoid_derivative(z1)
    dz1_dw11 = x[0]
    dz1_dw12 = x[1]
    dz1_db1 = 1
    
    da2_dz2 = sigmoid_derivative(z2)
    dz2_dw21 = x[0]
    dz2_dw22 = x[1]
    dz2_db2 = 1
    
    df_dw31 = df_dz3 * dz3_dw31
    df_dw32 = df_dz3 * dz3_dw32
    df_db3 = df_dz3 * dz3_db3
    
    df_dw11 = df_dz3 * dz3_dw31 * da1_dz1 * dz1_dw11
    df_dw12 = df_dz3 * dz3_dw31 * da1_dz1 * dz1_dw12
    df_db1 = df_dz3 * dz3_dw31 * da1_dz1 * dz1_db1
    
    df_dw21 = df_dz3 * dz3_dw32 * da2_dz2 * dz2_dw21
    df_dw22 = df_dz3 * dz3_dw32 * da2_dz2 * dz2_dw22
    df_db2 = df_dz3 * dz3_dw32 * da2_dz2 * dz2_db2
    
    return np.array([df_dw11, df_dw12, df_dw21, df_dw22, df_dw31, df_dw32]), np.array([df_db1, df_db2, df_db3])



In [123]:
def hessian_f(x, w, b):
    w11, w12, w21, w22, w31, w32 = w
    b1, b2, b3 = b
    
    z1 = w11*x[0] + w12*x[1] + b1
    z2 = w21*x[0] + w22*x[1] + b2
    a1 = sigmoid(z1)
    a2 = sigmoid(z2)
    
    df_dz3 = sigmoid_derivative(w31*a1 + w32*a2 + b3)
    da1_dz1 = sigmoid_derivative(z1)
    da2_dz2 = sigmoid_derivative(z2)
    
    # Initialize Hessian matrix
    hessian = np.zeros((len(x), len(x)))
    
    # Compute individual elements of the Hessian matrix
    hessian[0, 0] = df_dz3 * da1_dz1**2 * w31**2 * sigmoid_derivative(z1) * (1 - sigmoid_derivative(z1)) * x[0]**2
    hessian[0, 1] = df_dz3 * da1_dz1 * da2_dz2 * w31 * w32 * sigmoid_derivative(z1) * sigmoid_derivative(z2) * x[0] * x[1]
    hessian[1, 0] = hessian[0, 1]
    hessian[1, 1] = df_dz3 * da2_dz2**2 * w32**2 * sigmoid_derivative(z2) * (1 - sigmoid_derivative(z2)) * x[1]**2
    
    return hessian


In [124]:
def modified_ldl_factorization(A, tol=1e-6):
    n, m = A.shape
    assert n == m
    assert np.allclose(A, A.T)
    L = np.eye(n)
    D = np.zeros((n, n))
    for i in range(n):
        D[i, i] = A[i, i]
        for k in range(i):
            D[i, i] -= L[i, k] ** 2 * D[k, k]
        D[i, i] += tol
        for j in range(i + 1, n):
            L[j, i] = A[j, i]
            for k in range(i):
                L[j, i] -= L[j, k] * L[i, k] * D[k, k]
            L[j, i] /= D[i, i]
    return L, D



In [125]:
def modified_ldl_newton_method(x0, w, b, tol=1e-5, max_iter=100):
    x = np.array(x0)
    for _ in range(max_iter):
        gradient, _ = gradient_f(x, w, b)
        hessian = hessian_f(x, w, b)
        # print(hessian.shape)
        L, D = modified_ldl_factorization(hessian)
        
        d = np.linalg.solve(L, -gradient)
        step = np.linalg.solve(L.T, d)
        x = x + step
        
        if np.linalg.norm(step) < tol:
            break
    
    return x


In [126]:
# Define parameters
w = [1, 1, 1, 1, 1, 1]  # Adjust these weights as needed
b = [1, 1, 1]  # Adjust these biases as needed

# Initial guess
x0 = [0, 0, 0, 0, 0, 0]

# Test the code
optimal_x = modified_ldl_newton_method(x0, w, b)
print("Optimal x:", optimal_x)
print("Optimal f(x):", f(optimal_x, w, b))

Optimal x: [ 0.          0.          0.          0.         -5.29182266 -5.29182266]
Optimal f(x): 0.9214430516601156


# PART 2

In [127]:
import numpy as np

# Define the function f(x)
def f(x):
    Q = np.array([[5, 2], [2, 1]])
    b = np.array([3, 1])
    c = 0
    return 0.5 * np.dot(x.T, np.dot(Q, x)) - np.dot(b, x) + c

# Define the gradient of f(x)
def gradient_f(x):
    Q = np.array([[5, 2], [2, 1]])
    b = np.array([3, 1])
    return np.dot(Q, x) - b

# Conjugate Gradient Method
def conjugate_gradient_method(x0, tol=1e-5, max_iter=1000):
    x = x0
    d = -gradient_f(x)  # Initial direction is negative gradient
    iter_count = 0
    
    while np.linalg.norm(gradient_f(x)) > tol and iter_count < max_iter:
        Q = np.array([[5, 2], [2, 1]])  # Define Q within the loop
        alpha = np.dot(gradient_f(x).T, gradient_f(x)) / np.dot(np.dot(d.T, Q), d)
        x = x + alpha * d
        beta = np.dot(gradient_f(x).T, gradient_f(x)) / np.dot(gradient_f(x - alpha * d).T, gradient_f(x - alpha * d))
        d = -gradient_f(x) + beta * d
        iter_count += 1
    
    return x

# Initial guess for x0
x0 = np.array([0, 0])

# Find the minimizer using conjugate gradient method
optimal_x = conjugate_gradient_method(x0)
optimal_f = f(optimal_x)

print("Optimal x:", optimal_x)
print("Optimal f(x):", optimal_f)


Optimal x: [ 1. -1.]
Optimal f(x): -1.0


# PART 4

In [128]:
import numpy as np

def conjugate_gradient(Q, b, x0, tol=1e-5, max_iter=None):
    n = Q.shape[0]
    if max_iter is None:
        max_iter = n
    
    x = x0
    r = b - np.dot(Q, x)
    p = r
    rsold = np.dot(r, r)
    
    for i in range(max_iter):
        Qp = np.dot(Q, p)
        alpha = rsold / np.dot(p, Qp)
        x = x + alpha * p
        r = r - alpha * Qp
        rsnew = np.dot(r, r)
        if np.sqrt(rsnew) < tol:
            break
        p = r + (rsnew / rsold) * p
        rsold = rsnew
    
    return x, i+1  # Return the solution and the number of iterations

# Define Q and b for testing
n = 5
a = np.ones(n)
Q = np.eye(n) + np.outer(a, a)
b = np.random.rand(n)
x0 = np.zeros(n)

# Solve using conjugate gradient method
solution, iterations = conjugate_gradient(Q, b, x0)

print("Solution:", solution)
print("Number of iterations:", iterations)


Solution: [ 0.13268401  0.30809564  0.08281502  0.45626555 -0.44534658]
Number of iterations: 2


# PART 5

In [131]:
import numpy as np

# Define the sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Define the function f(x1, x2)
def f(x1, x2, w, b):
    z1 = sigmoid(w[0, 0]*x1 + w[0, 1]*x2 + b[0])
    z2 = sigmoid(w[1, 0]*x1 + w[1, 1]*x2 + b[1])
    return sigmoid(w[2, 0]*z1 + w[2, 1]*z2 + b[2])

# Define the gradient of f(x1, x2) with respect to x1 and x2
def gradient_f(x1, x2, w, b):
    z1 = sigmoid(w[0, 0]*x1 + w[0, 1]*x2 + b[0])
    z2 = sigmoid(w[1, 0]*x1 + w[1, 1]*x2 + b[1])
    z3 = sigmoid(w[2, 0]*z1 + w[2, 1]*z2 + b[2])
    dz1_dw11 = x1 * z1 * (1 - z1)
    dz1_dw12 = x2 * z1 * (1 - z1)
    dz2_dw21 = x1 * z2 * (1 - z2)
    dz2_dw22 = x2 * z2 * (1 - z2)
    dz3_dw31 = z1 * z3 * (1 - z3)
    dz3_dw32 = z2 * z3 * (1 - z3)
    return np.array([
        (w[2, 0] * dz3_dw31 * dz1_dw11 + w[2, 1] * dz3_dw32 * dz2_dw21),
        (w[2, 0] * dz3_dw31 * dz1_dw12 + w[2, 1] * dz3_dw32 * dz2_dw22)
    ])

# Define the Hessian of f(x1, x2) with respect to x1 and x2
def hessian_f(x1, x2, w, b):
    z1 = sigmoid(w[0, 0]*x1 + w[0, 1]*x2 + b[0])
    z2 = sigmoid(w[1, 0]*x1 + w[1, 1]*x2 + b[1])
    z3 = sigmoid(w[2, 0]*z1 + w[2, 1]*z2 + b[2])
    dz1_dw11 = x1 * z1 * (1 - z1)
    dz1_dw12 = x2 * z1 * (1 - z1)
    dz2_dw21 = x1 * z2 * (1 - z2)
    dz2_dw22 = x2 * z2 * (1 - z2)
    dz3_dw31 = z1 * z3 * (1 - z3)
    dz3_dw32 = z2 * z3 * (1 - z3)
    return np.array([
        [w[2, 0]**2 * dz3_dw31 * (1 - 2*z3) * dz1_dw11**2 + w[2, 1]**2 * dz3_dw32 * (1 - 2*z3) * dz2_dw21**2,
         w[2, 0] * w[2, 1] * dz3_dw31 * dz1_dw11 * dz3_dw32 * dz2_dw21],
        [w[2, 0] * w[2, 1] * dz3_dw31 * dz1_dw12 * dz3_dw32 * dz2_dw22,
         w[2, 0]**2 * dz3_dw31 * (1 - 2*z3) * dz1_dw12**2 + w[2, 1]**2 * dz3_dw32 * (1 - 2*z3) * dz2_dw22**2]
    ])

# Define the modified LDL Newton method
def modified_ldl_newton_method(x0, w, b, tol=1e-5, max_iter=1000):
    x = x0
    for _ in range(max_iter):
        gradient = gradient_f(x[0], x[1], w, b)
        hessian = hessian_f(x[0], x[1], w, b)
        L, D = modified_ldl_factorization(hessian)
        d = np.linalg.solve(L, -gradient)
        step = np.linalg.solve(L.T, d)
        x = x + step
        if np.linalg.norm(step) < tol:
            break
    return x

# Define the DFP method
def dfp_method(x0, w, b, tol=1e-5, max_iter=1000):
    x = x0
    H = np.eye(2)  # Initialize the Hessian approximation
    for _ in range(max_iter):
        gradient = gradient_f(x[0], x[1], w, b)
        step = -np.dot(H, gradient)
        alpha = 0.01  # Choose a step size
        x_new = x + alpha * step
        s = x_new - x
        y = gradient_f(x_new[0], x_new[1], w, b) - gradient
        rho = 1 / np.dot(y, s)
        A = np.eye(2) - rho * np.outer(s, y)
        B = np.eye(2) - rho * np.outer(y, s)
        H = np.dot(np.dot(A, H), B) + rho * np.outer(s, s)
        x = x_new
        if np.linalg.norm(step) < tol:
            break
    return x

# Test the code with given function and parameter values
x0 = np.array([0, 0])
w = np.array([[1, 2], [3, 4], [5, 6]])
b = np.array([1, 1, 1])


In [132]:
# Define the DFP method with adjusted step size and iteration count
def dfp_method(x0, w, b, tol=1e-5, max_iter=1000):
    x = x0
    H = np.eye(2)  # Initialize the Hessian approximation
    iter_count = 0  # Initialize iteration count
    for _ in range(max_iter):
        gradient = gradient_f(x[0], x[1], w, b)
        step = -np.dot(H, gradient)
        alpha = 0.001  # Adjusted step size
        perturbation = 1e-8  # Perturbation value
        x_new = x + alpha * step + perturbation * np.random.randn(2)  # Add perturbation
        s = x_new - x
        y = gradient_f(x_new[0], x_new[1], w, b) - gradient
        sy = np.dot(s, y)
        if sy == 0:
            continue  # Skip update if sy is zero
        rho = 1 / sy
        A = np.eye(2) - rho * np.outer(s, y)
        B = np.eye(2) - rho * np.outer(y, s)
        H = np.dot(np.dot(A, H), B) + rho * np.outer(s, s)
        x = x_new
        iter_count += 1
        if np.linalg.norm(step) < tol:
            break
    return x, iter_count

# Test the code with given function and parameter values
x0 = np.array([0, 0])
w = np.array([[1, 2], [3, 4], [5, 6]])
b = np.array([1, 1, 1])

optimal_x_ldl = modified_ldl_newton_method(x0, w, b)

optimal_x_dfp, iter_count_dfp = dfp_method(x0, w, b)
print("Optimal x (Modified LDL Newton Method):", optimal_x_ldl)
print("Optimal x (DFP Method):", optimal_x_dfp)
print("Number of iterations for DFP Method:", iter_count_dfp)


Optimal x (Modified LDL Newton Method): [0. 0.]
Optimal x (DFP Method): [ 1.62890565e-08 -2.63914486e-09]
Number of iterations for DFP Method: 1
